In [ ]:
import os
import gc
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as sio
from scipy.signal import resample
import warnings

# Weights and Biases related imports
#import wandb
#from wandb.integration.keras import WandbMetricsLogger
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# 警告を無視する設定
warnings.simplefilter("ignore")

# Pandasの表示設定
pd.set_option('display.max_columns', None)

# Jupyter Notebookでのグラフ表示を有効にする
#%matplotlib inline

# Weights and Biasesにログイン
#wandb.login()

前処理

In [ ]:
# 筋電図データの整流化
def rectify_signal(emg_data):
    """EMGデータを整流化（負の値を正に変換）"""
    return np.abs(emg_data)

# 筋電位データの対数変換関数
def log_transform_emg(emg_data, epsilon=1e-6):
    """整流化されたEMGデータに対して対数変換を適用"""
    return np.log(emg_data + epsilon)

# x方向が常に正または負のデータを抽出
def extract_positive_x_trials(velocity_x):
    positive_x_trials = []
    for i in range(velocity_x.shape[0]):
        if np.all(velocity_x[i] > 0) or np.all(velocity_x[i] < 0):
            positive_x_trials.append(i)
    return positive_x_trials

# 筋電位データの左右を反転して学習データに追加
def add_reversed_emg_data(trial_user, x_velocity, emg_data, y_data):
    added_user_list = []
    reversed_emg_data = []
    y_add_list = []
    
    # 各ユーザーごとに処理
    user_trial_ranges = np.cumsum([0] + trial_user)
    right_leg_muscles = [1, 3, 5, 7, 9, 11, 13, 15]
    left_leg_muscles = [0, 2, 4, 6, 8, 10, 12, 14]

    for user_idx, (start, end) in enumerate(zip(user_trial_ranges[:-1], user_trial_ranges[1:])):
        user_velocity_x = x_velocity[start:end]
        user_emg_data = emg_data[start:end]
        user_y_data = y_data[start:end]

        positive_x_trials = extract_positive_x_trials(user_velocity_x)

        for i in positive_x_trials:
            trial_emg = user_emg_data[i].copy()
            y_add = user_y_data[i].copy()

            # 筋電位データの左右を反転
            trial_emg[:, right_leg_muscles], trial_emg[:, left_leg_muscles] = trial_emg[:, left_leg_muscles], trial_emg[:, right_leg_muscles]
            
            reversed_emg_data.append(trial_emg)
            y_add_list.append(y_add)
            added_user_list.append(f'00{user_idx + 1}')  # ユーザーIDを追加

    reversed_emg_data = np.array(reversed_emg_data)
    y_add_data = np.array(y_add_list)

    new_emg_data = np.concatenate([emg_data, reversed_emg_data], axis=0)
    new_y_data = np.concatenate([y_data, y_add_data], axis=0)

    return new_emg_data, added_user_list, new_y_data

# 学習用データの取得
def get_train_data(train_path, user_ids, trial_user):
    train_data = sio.loadmat(train_path)
    X_list, y_list, user_id_list = [], [], []
    user_id_encoder = OneHotEncoder(sparse_output=False)
    user_id_encoder.fit(user_ids.reshape(-1, 1))

    for user_id in user_ids:
        x_array = train_data[user_id][0][0][0].transpose(0, 2, 1)
        y_array = train_data[user_id][0][0][1].transpose(0, 2, 1)
        num_trials = x_array.shape[0]
        
        user_id_encoded = user_id_encoder.transform(np.array([[user_id]] * num_trials))
        user_id_list.extend([[user_id]] * num_trials)

        for trial in range(num_trials):
            rectified_x = rectify_signal(x_array[trial])
            log_transformed_x = log_transform_emg(rectified_x)
            new_x_array = np.hstack([log_transformed_x, np.repeat(user_id_encoded[trial][np.newaxis, :], 1000, axis=0)])
            X_list.append(new_x_array[np.newaxis, :])

        y_list.append(y_array)
        del x_array, y_array
        gc.collect()

    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)

    print(X.shape)

    # 標準化
    X_scaled, scaler_list = standardize_by_user(X, trial_user)
    x_velocity = y[:,:,0]
    new_emg_data, added_user_list, y_add_data = add_reversed_emg_data(trial_user, x_velocity, X_scaled, y)

    # user_id_listに追加されたデータのuser_idを追加
    user_id_list_flat = [str(item[0]) if isinstance(item, np.ndarray) else str(item) for sublist in user_id_list for item in sublist]
    user_id_list_flat.extend(added_user_list)
    user_id_list_array = np.array(user_id_list_flat)

    return new_emg_data, y_add_data, user_id_list_array, scaler_list

# ユーザーごとの標準化
def standardize_by_user(emg_data, trial_user):
    scaler_list = []
    start = 0
    for num_trials in trial_user:
        end = start + num_trials# * 1000
        scaler = StandardScaler()
        emg_data[start:end, :, :16] = scaler.fit_transform(emg_data[start:end, :, :16].reshape(-1, 16)).reshape(num_trials, 1000, 16)
        scaler_list.append(scaler)
        start = end
    return emg_data[:, :-10], scaler_list

# 評価用データの取得
def get_test_data(test_path, user_ids, trial_user, scaler_list):
    test_data = sio.loadmat(test_path)
    X_list, user_id_list = [], []
    user_id_encoder = OneHotEncoder(sparse_output=False)
    user_id_encoder.fit(user_ids.reshape(-1, 1))

    for user_id in user_ids:
        x_array = test_data[user_id][0][0][0].transpose(0, 2, 1)
        num_trials = x_array.shape[0]
        
        user_id_encoded = user_id_encoder.transform(np.array([[user_id]] * num_trials))
        user_id_list.extend([[user_id]] * num_trials)

        for trial in range(num_trials):
            rectified_x = rectify_signal(x_array[trial])
            log_transformed_x = log_transform_emg(rectified_x)
            new_x_array = np.hstack([log_transformed_x, np.repeat(user_id_encoded[trial][np.newaxis, :], 1000, axis=0)])
            X_list.append(new_x_array[np.newaxis, :])

        del x_array
        gc.collect()

    X = np.concatenate(X_list, axis=0)

    # ユーザーごとに標準化
    X_scaled = np.zeros_like(X)
    start = 0
    for user_idx, num_trials in enumerate(trial_user):
        end = start + num_trials
        X_scaled[start:end, :, :16] = scaler_list[user_idx].transform(X[start:end, :, :16].reshape(-1, 16)).reshape(num_trials, 1000, 16)
        start = end

    return X_scaled[:, :-10]


In [47]:
train_path = r'C:\Users\sato\Desktop\python\SIGNATE\スケートボードの挙動予測\input/train.mat'
test_path = r'C:\Users\sato\Desktop\python\SIGNATE\スケートボードの挙動予測\input/test.mat'
user_ids = np.array(["0001", "0002", "0003", "0004"])
trial_user = [319, 300, 320, 320] 

train_X, y, user_id_list, scaler_list = get_train_data(train_path, user_ids, trial_user)
test_X = get_test_data(test_path, user_ids, trial_user, scaler_list)

(1259, 1000, 20)


学習

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Bidirectional, Input, TimeDistributed, Attention, AveragePooling1D, BatchNormalization, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import root_mean_squared_error
#import wandb
#from wandb.integration.keras import WandbMetricsLogger
import numpy as np

In [ ]:
def create_model():
    # モデル定義
    input_layer = Input(shape=(990, 20))

    # 畳み込み層
    x = Conv1D(filters=224, kernel_size=50, activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = AveragePooling1D(pool_size=4)(x)
    x = Conv1D(filters=128, kernel_size=10, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = AveragePooling1D(pool_size=4)(x)
    x = MaxPooling1D(pool_size=2)(x)

    # LSTM + Attention層
    x = Bidirectional(LSTM(96, return_sequences=True))(x)
    x = Dropout(rate=0.3)(x)

    # Attention層
    attention_out = Attention()([x, x]) 

    # LSTM層と出力層
    x = Bidirectional(LSTM(64, return_sequences=True))(attention_out)
    x = Dropout(rate=0.2)(x)

    # 出力層
    output_layer = TimeDistributed(Dense(3))(x)

    # モデルの構築
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

def rmse_3d_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1))

def train_model(X_train, y_train, user_id_list, num_folds=10):
    kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)
    model_list = []
  
    for i, (train_idx, val_idx) in enumerate(kf.split(X_train, user_id_list.astype('str'))):
        #run = wandb.init(
        #    project = "sgnate_skate-keras",
        #    name = f'test_{i}',
        #    tags = [f'test_up']
        #    #config = configs
        #)
        #wandb.define_metric('rmase_prediction', summary='mean')
        train_X, val_X = X_train[train_idx], X_train[val_idx]
        train_y, val_y = y_train[train_idx], y_train[val_idx]
        
        model = create_model()
        model.compile(optimizer=Adam(learning_rate=0.0004926459300637485), loss=rmse_3d_loss) #過去コンペで使用したlearning_rateを使用
        early_stopping = EarlyStopping(patience=35, restore_best_weights=True, verbose=1)
        
        #model.fit(train_X, train_y, validation_data=(val_X, val_y), epochs=1000, batch_size=32, callbacks=[early_stopping, WandbMetricsLogger()])
        model.fit(train_X, train_y, validation_data=(val_X, val_y), epochs=1000, batch_size=32, callbacks=[early_stopping])
        #model.save(f'model_fold_{i+1}.h5')
        model_list.append(model)

        pred = model.predict(val_X)
        score = root_mean_squared_error(val_y.reshape(-1, 1), pred.reshape(-1, 1))
        print(score)
        #wandb.log({'rmase_prediction': score})
        
        #run.finish()

    return model_list

In [ ]:
model_list = train_model(train_X, y, user_id_list)

予測

In [71]:
def predict(model_list, test_X):
    preds_test = []
    for model in model_list:
        pred_test = model.predict(test_X)
        preds_test.append(pred_test)

    pred_test = np.mean(preds_test, axis=0) 
    #予測値の反転
    pred_test[:, :, 0] = pred_test[:, :, 0] * -1
    pred_test[:, :, 1] = pred_test[:, :, 1] * -1

    return pred_test

In [72]:
pred_test = predict(model_list, test_X)

40/40 [==============================] - 0s 11ms/step


提出用ファイルを作成

In [74]:
# サンプル提出ファイルを確認します
with open(r"C:\Users\sato\Desktop\python\SIGNATE\スケートボードの挙動予測\input\sample_submit.json") as r:
    sample_submit = json.load(r)

In [75]:
# 提出用データの作成 (vel_x, vel_y, vel_z の形式に整形)
user_id_sub = sample_submit.keys()
trial_user = [319, 300, 320, 320]  # 各ユーザーのトライアル数

test_index_start = 0
test_index_end = 0

for i, user_id in enumerate(user_id_sub):
    sub_dict = {}
    trial_num = trial_user[i]
    test_index_start = test_index_end
    test_index_end += trial_num

    # テストデータの予測結果から各ユーザーごとのデータを抽出
    sub_pred = pred_test[test_index_start:test_index_end]#.reshape(-1, 30, 3)
    print(f"User {user_id} predictions shape: {sub_pred.shape}")  # (trial_num, 30, 3) の形になることを確認

     # 各トライアルの予測データを辞書形式に整形しつつ、RMSEを計算
    for trial in range(trial_num):
        trial_ = trial + 1
        sub_dict[f"trial{trial_}"] = sub_pred[trial].tolist()  # 各トライアルのデータをリスト形式に変換

# 提出用辞書に保存
    sample_submit[f"sub{i+1}"] = sub_dict

User sub1 predictions shape: (319, 30, 3)
User sub2 predictions shape: (300, 30, 3)
User sub3 predictions shape: (320, 30, 3)
User sub4 predictions shape: (320, 30, 3)


In [76]:
# 提出ファイルとして保存します
with open(r"C:\Users\sato\Desktop\python\SIGNATE\スケートボードの挙動予測\submission\submission_.json", "w") as f:
    json.dump(sample_submit, f)